In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import random

In [5]:
# Selenium 설정
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저를 띄우지 않고 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [6]:
# 알리 베스트셀러 url 접속
driver.get("https://www.aliexpress.com/gcp/300001064/aptaxwR82N?spm=a2g0o.best.testStatic.3.322e423aWOTh7f&disableNav=YES&pha_manifest=ssr&_immersiveMode=true")
# 창크기 최대로
driver.maximize_window()
# 명시적 대기 설정
wait = WebDriverWait(driver, 5)
time.sleep(5)
# 상단베너 검색
상단베너 = driver.find_elements(By.CSS_SELECTOR, 'div.rax-view-v2.tab_pc_content_item')
# 스포츠 및 엔터테인먼트 베너를 찾아서 클릭
for 베너 in 상단베너:
    if "스포츠 및 엔터테인먼트" in 베너.text:
        driver.execute_script("arguments[0].click();", 베너)
        break
# 추가 작업을 위해 잠시 대기
time.sleep(5)

# 화면크기를 줄여 많은 상품이 나오게 축소
driver.execute_script("document.body.style.zoom='25%'")
time.sleep(5)


In [7]:
items = []
last_height = driver.execute_script("return document.body.scrollHeight")

# 50개의 항목이 존재할때 까지 페이지 스크롤을 내림(동적 페이지)
while len(items) < 50:
    #PgDn키 입력 (50개의 항목이 존재할 떄 까지)
    # body = driver.find_element(By.TAG_NAME, 'body')
    # body.send_keys(Keys.PAGE_DOWN)
    #스크롤내리고 제품이 뜨기까지 딜레이가 있어보여 5초간 대기
    # time.sleep(random.uniform(2, 4))
    # class="swiper-slide swiper-slide-active" 요소 찾기
    swiper_slide = driver.find_element(By.CLASS_NAME, "swiper-slide.swiper-slide-active")
    # 하위의 class="productContainer" 요소들 찾기
    items = swiper_slide.find_elements(By.CLASS_NAME, "productContainer")
    # 새로운 높이를 계산하여 이전 높이와 비교
    new_height = driver.execute_script("return document.body.scrollHeight")
    # 높이가 변하지 않으면 더 이상 항목이 로드되지 않음을 의미
    if new_height == last_height:
        print("더 이상 항목이 로드되지 않음")
        break
    
    last_height = new_height

print(f"총 로드된 항목 수: {len(items)}")
df = pd.DataFrame({"상품명": [], '가격': [], '평점': [], '댓글': [], '별점': [], '링크': []})

더 이상 항목이 로드되지 않음
총 로드된 항목 수: 60


In [8]:
for ii, item in enumerate(items):
    # 요소가 화면에 보이도록 스크롤
    driver.execute_script("arguments[0].scrollIntoView();", item)
    # 자바스크립트를 사용하여 클릭
    driver.execute_script("arguments[0].click();", item)
    # 마지막 탭으로 이동
    last_tab = driver.window_handles[-1]
    driver.switch_to.window(window_name=last_tab)

    # 상품정보 파싱
    상품명 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".title--wrap--Ms9Zv4A > h1"))).text
    평점 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.reviewer--wrap--sPGWrNq > strong"))).text.strip()
    가격 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product-price-current"))).text[1:]
    링크 = driver.current_url  # 상품 링크 수집
    
    time.sleep(5)
    # 댓글 및 별점수집
    댓글 = []
    별점 = []
    
    # 화면크기를 줄여 많은 상품이 나오게 축소
    # driver.execute_script("document.body.style.zoom='25%'")    
    try:
            #end키 입력
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.END)
            # 댓글 더보기 버튼 클릭
            더보기_버튼 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pdp-body-top-left .review--wrap--yhZbZox .ae-evaluation-list .comet-v2-btn.comet-v2-btn-slim.comet-v2-btn-large.comet-v2-btn-important")))
            # 더보기 버튼이 화면에 보이도록 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", 더보기_버튼)
            driver.execute_script("arguments[0].click();", 더보기_버튼)
            time.sleep(2)
            
            # 원래 번역 표시 버튼 클릭
            try:
                원래번역표시 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.ae-all-list-content .ae-filter-select-box .ae-filter-select-translate')))
                원래번역표시.click()
                time.sleep(2)  # 버튼 클릭 후 잠시 대기
            except Exception as e:
                print(f"원래 번역 표시 버튼 클릭 실패: {e}")
    except:
            pass
        
    while len(댓글) < 100:
        # try:
        #     #end키 입력
        #     body = driver.find_element(By.TAG_NAME, 'body')
        #     body.send_keys(Keys.END)
        #     # 댓글 더보기 버튼 클릭
        #     더보기_버튼 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".pdp-body-top-left .review--wrap--yhZbZox .ae-evaluation-list .comet-v2-btn.comet-v2-btn-slim.comet-v2-btn-large.comet-v2-btn-important")))
        #     # 더보기 버튼이 화면에 보이도록 스크롤
        #     driver.execute_script("arguments[0].scrollIntoView();", 더보기_버튼)
        #     driver.execute_script("arguments[0].click();", 더보기_버튼)
        #     time.sleep(2)
            
        #     # 원래 번역 표시 버튼 클릭
        #     try:
        #         원래번역표시 = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.ae-all-list-content .ae-filter-select-box .ae-filter-select-translate')))
        #         원래번역표시.click()
        #         time.sleep(2)  # 버튼 클릭 후 잠시 대기
        #     except Exception as e:
        #         print(f"원래 번역 표시 버튼 클릭 실패: {e}")
            
        try:        
            # 팝업 창의 스크롤 영역 식별
            popup_scroll_area = driver.find_element(By.CSS_SELECTOR, "div.ae-all-list-content")
                
            # 팝업 창 스크롤하여 댓글 추가 수집
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", popup_scroll_area)
            time.sleep(4)
        
        except:
            pass
            
        # 댓글 및 별점 파싱
        댓글_elements = driver.find_elements(By.CSS_SELECTOR, ".ae-evaluateList-box .ae-evaluateList-card-content")
        별점_elements = driver.find_elements(By.CSS_SELECTOR, ".ae-evaluateList-box .ae-stars-box")
            
        # 새로 수집된 댓글과 별점 수집
        new_댓글 = [element.text for element in 댓글_elements if any('\u3131' <= char <= '\u3163' or '\uac00' <= char <= '\ud7a3' for char in element.text)]
        # 별점 이미지 개수 파악하여 별점 계산
        new_별점 = []
        for star_box in 별점_elements:
            star_images = star_box.find_elements(By.CSS_SELECTOR, "img.ae-stars")
            star_count = sum(1 for img in star_images if "S3a90fde0c9cc4b8c8958a6ee7553e26bf/42x42.png" in img.get_attribute("src"))
            new_별점.append(str(star_count))  # 별점 숫자로 변환하여 저장
            
        # 새로운 댓글과 별점이 이미 수집된 항목과 중복되지 않도록 필터링
        unique_댓글 = [item for item in new_댓글 if item not in 댓글]
        unique_별점 = [item for item in new_별점 if item not in 별점]
            
        # 수집한 댓글과 별점 추가
        댓글.extend(unique_댓글)
        별점.extend(unique_별점)
            
        # 수집된 댓글이 100개를 넘으면 루프 종료
        if len(댓글) >= 100:
            break
            
        # 더 이상 새로운 댓글이 없으면 루프 종료
        if not unique_댓글:
            break
            
    # 100개 이상 수집된 경우 100개로 자르기
    댓글 = 댓글[:100]
    별점 = 별점[:100]

    # 데이터프레임에 추가
    df.loc[ii] = [상품명, 가격, 평점, '\n'.join(댓글), '\n'.join(별점), 링크]
        
    # 현재 탭 닫기
    driver.close()

    # 메인탭으로 이동
    main_tab = driver.window_handles[0]
    driver.switch_to.window(window_name=main_tab)

# driver 종료
driver.quit()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   상품명     60 non-null     object
 1   가격      60 non-null     object
 2   평점      60 non-null     object
 3   댓글      60 non-null     object
 4   별점      60 non-null     object
 5   링크      60 non-null     object
dtypes: object(6)
memory usage: 3.3+ KB


In [10]:
df.to_csv('스포츠 및 엔터테인먼트03.csv',index=False,encoding='utf-8-sig')

In [11]:
df


,상품명,가격,평점,댓글,별점,링크
0,"SCVCN 사이클링 안경 자전거 선글라스, 남성 UV400 안경, 스포츠 MTB 야...","1,391",4.7,,,https://ko.aliexpress.com/item/100500592463696...
1,"역도 스트랩 실리콘 리프팅 손목 스트랩, 미끄럼 방지, 힘 훈련, 데드리프트 크로스...","1,391",4.8,배송은 생각보다 빨리도착했네요 손목 스트랩 1쌍~레드 디자인 이쁘고 마감처리 깔끔하...,5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n5...,https://ko.aliexpress.com/item/100500592457862...
2,"야외 캠핑 생존 랜야드 낙하산 로프 하이킹 텐트 액세서리, 7 코어 550 파라코드...","1,125",4.7,감사합니다\n지퍼 손잡이 매듭용으로 좋습니다\n잘받았습니다 잘받았습니다\n이색인줄은...,5\n5\n5\n5\n5\n5\n5\n3\n5\n5\n5\n5\n4\n5\n4\n5...,https://ko.aliexpress.com/item/100500522141320...
3,"접이식 대용량 접이식 왜건, 휴대용 유틸리티 카트, 쇼핑 비치 가든 풀 트롤리, 야...","25,785",4.7,뭔 배송하면서 바닷물에 담궈서 가져오냐? 쓰기도 전에 녹슬어서 오는 메이드인 차이나...,5\n5\n5\n5\n5\n5\n4\n4\n5\n5\n5\n5\n5\n5\n5\n4...,https://ko.aliexpress.com/item/100500593200533...
4,"BISINNA 접이식 바베큐 라운드 테이블 스토브, 휴대용 캠핑 바베큐 숯 그릴, ...","16,411",4.8,생각보다 너무괜찮은데요? 후기좋은 이유가있엇네요 불판도생각보다 두껍고튼튼한듯ㅎ 저사...,5\n5\n5\n5\n5\n5\n5\n5\n5\n4\n5\n5\n5\n5\n5\n5...,https://ko.aliexpress.com/item/100500626084886...
5,"튼튼한 라텍스 저항 밴드, 탄성 운동 강도 풀업 보조 밴드, 필라테스 체육관 피트니...","1,391",4.7,잘받았습니다\n좋아요 가격대비 만족합니다\n진짜 빨리받앗어요. 일주일?? 턱걸이하는...,5\n5\n5\n5\n5\n5\n5\n5\n5\n4\n5\n4\n4\n4\n4\n5...,https://ko.aliexpress.com/item/100500545860164...
6,"남성 및 여성용 사이클링 반 손가락 체육관 장갑, 미끄럼 방지 통기성 장갑, 피트니...","1,391",4.6,가격은 한국과 비슷한데 퀄리티가 아주좋아요 오랜만에 맘에 드는 상품을 구입한것같아요...,5\n5\n5\n5\n5\n5\n5\n5\n5\n5\n4\n2\n5\n4\n5\n5...,https://ko.aliexpress.com/item/100500601428368...
7,"USB 충전식 자전거 전조등 세트, 미등 포함, 쉬운 설치, 3 가지 모드, MTB...","1,391",4.5,크기가 성인 남석 새끼손가락 길이 정도 되네요. 배터리가 얼마나 갈지 모르겠습니다만...,5\n5\n5\n5\n5\n5\n5\n5\n4\n4\n3\n2\n2\n5\n4\n5...,https://ko.aliexpress.com/item/100500599945778...
8,"스포츠 방수 운동 요법 테이프 스티커, 운동 붕대, 근육 회복 보호대, 스트랩 통증...","1,391",4.7,아직 사용 전입니다 . 쫀쫀하고 접착력도 좋습니다. 색상도 다양하고 좋습니다. 다용...,5\n5\n4\n5\n5\n4\n5\n5\n5\n4\n5\n5\n5\n5\n4\n4...,https://ko.aliexpress.com/item/100500607741586...
9,"실리콘 턱 운동기 얼굴 토너 및 턱 라인 피트니스 공, 목 토닝 장비, 얼굴 미용 ...","1,222",4.6,매일 열심히하면 좋아질듯함.\n입이 너무아파요\n맛 없어서 버렸어요,5\n5\n3\n5\n5\n3\n5\n5\n5\n5\n5\n5\n5\n5\n4\n5...,https://ko.aliexpress.com/item/100500563705697...
